# Analysere ER for å finne riktige variabler å etterligne

Har lastet ned ER fra https://hotell.difi.no/download/brreg/enhetsregisteret?download og lagret som **er.csv**.

Kan ikke ha IDer (funker dårlig pluss unyttig).

In [1]:
import pandas as pd

ImportError: No module named 'requests'

In [6]:
!apt -y update && apt -y install curl && apt -y install wget

Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]               
Ign:3 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Ign:4 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:5 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release [564 B]
Get:6 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release [564 B]
Get:7 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release.gpg [801 B]
Get:8 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release.gpg [801 B]
Get:9 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Packages [136 kB]
Get:10 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Packages [25.2 kB]
Get:11 http://archi

In [7]:
!curl  https://hotell.difi.no/download/brreg/enhetsregisteret?download > /storage/er.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  237M    0  237M    0     0   667k      0 --:--:--  0:06:03 --:--:-- 5899k 173M    0     0   508k      0 --:--:--  0:05:49 --:--:-- 4346k


In [8]:
er = pd.read_csv('/storage/er.csv', sep=";")

In [9]:
er.dtypes

orgnr                  int64
navn                  object
organisasjonsform     object
forretningsadr        object
forradrpostnr        float64
forradrpoststed       object
forradrkommnr        float64
forradrkommnavn       object
forradrland           object
postadresse           object
ppostnr              float64
ppoststed             object
ppostland             object
regifr                object
regimva               object
nkode1               float64
nkode2               float64
nkode3               float64
sektorkode           float64
konkurs               object
avvikling             object
tvangsavvikling       object
regiaa                object
regifriv              object
regdato               object
stiftelsesdato        object
tlf                   object
tlf_mobil             object
url                   object
regnskap             float64
hovedenhet           float64
ansatte_antall       float64
ansatte_dato          object
dtype: object

In [10]:
er.shape

(1098180, 33)

In [11]:
er.loc[:, ["nkode1", "forradrkommnr", "sektorkode", "ansatte_antall", "regdato"]].tail()

,nkode1,forradrkommnr,sektorkode,ansatte_antall,regdato
1098175,41.109,1865.0,2100.0,NaN,03.10.2009
1098176,96.090,937.0,7000.0,NaN,27.04.2015
1098177,86.909,1246.0,2100.0,1.0,10.03.2015
1098178,68.209,220.0,2100.0,NaN,30.03.2016
1098179,90.020,301.0,8200.0,NaN,09.10.2015


33 variabler, mesteparten unødvendig. Viktige ting å ha med:
- Næringskode (nkode1)
- kommune (forradrkommnr)
- sektor (sektorkode)
- Antall ansatte (ansatte_antall)
- Stiftelsesdato (grunnet NaN bruker heller regdato, bonus: mindre entropi)


Må også gjøre om en del datatyper.

Leser inn på nytt.

In [13]:
import numpy as np
dtypes = {'nkode1': 'str', 'forradrkommnr': 'str', 'sektorkode': 'str', 'regdato': 'str' }
parse_dates = ['regdato']

prepare = pd.read_csv('/storage/er.csv', sep=";", dtype=dtypes, parse_dates=parse_dates).query('ansatte_antall>0')

In [14]:
prepare.dtypes

orgnr                         int64
navn                         object
organisasjonsform            object
forretningsadr               object
forradrpostnr               float64
forradrpoststed              object
forradrkommnr                object
forradrkommnavn              object
forradrland                  object
postadresse                  object
ppostnr                     float64
ppoststed                    object
ppostland                    object
regifr                       object
regimva                      object
nkode1                       object
nkode2                      float64
nkode3                      float64
sektorkode                   object
konkurs                      object
avvikling                    object
tvangsavvikling              object
regiaa                       object
regifriv                     object
regdato              datetime64[ns]
stiftelsesdato               object
tlf                          object
tlf_mobil                   

In [15]:
prepare.loc[:, ["nkode1", "forradrkommnr", "sektorkode", "ansatte_antall", "regdato"]].tail()

,nkode1,forradrkommnr,sektorkode,ansatte_antall,regdato
1098152,47.111,0701,8200,4.0,1995-02-19
1098155,68.209,0301,2100,5.0,1995-02-20
1098161,68.201,0104,8300,10.0,1995-02-19
1098168,68.201,5001,8300,3.0,2006-10-24
1098177,86.909,1246,2100,1.0,2015-10-03


Need dummies for:
- nkode1
- forradrkommnr
- sektorkode

instead of dummies, tokenize? Nah. Start with dummies.

In [16]:
dummies = {}
for arg in ["nkode1", "forradrkommnr", "sektorkode"]:
    dummies[arg] = pd.get_dummies(prepare[arg], prefix=arg)

In [17]:
dummies.keys()

dict_keys(['forradrkommnr', 'nkode1', 'sektorkode'])

In [18]:
dummies["forradrkommnr"].head()

,forradrkommnr_0101,forradrkommnr_0104,forradrkommnr_0105,forradrkommnr_0106,forradrkommnr_0111,forradrkommnr_0118,forradrkommnr_0119,forradrkommnr_0121,forradrkommnr_0122,forradrkommnr_0123,...,forradrkommnr_5045,forradrkommnr_5046,forradrkommnr_5047,forradrkommnr_5048,forradrkommnr_5049,forradrkommnr_5050,forradrkommnr_5051,forradrkommnr_5052,forradrkommnr_5053,forradrkommnr_5054
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df = prepare.loc[:, ["nkode1", "forradrkommnr", "sektorkode", "ansatte_antall", "regdato"]]

In [20]:
df.drop(dummies.keys(), axis=1, inplace=True)

In [21]:
for key in dummies.keys():
    df = df.join(dummies[key])

In [22]:
df.shape

(173447, 1231)

In [23]:
df.dtypes

ansatte_antall               float64
regdato               datetime64[ns]
forradrkommnr_0101             uint8
forradrkommnr_0104             uint8
forradrkommnr_0105             uint8
forradrkommnr_0106             uint8
forradrkommnr_0111             uint8
forradrkommnr_0118             uint8
forradrkommnr_0119             uint8
forradrkommnr_0121             uint8
forradrkommnr_0122             uint8
forradrkommnr_0123             uint8
forradrkommnr_0124             uint8
forradrkommnr_0125             uint8
forradrkommnr_0127             uint8
forradrkommnr_0128             uint8
forradrkommnr_0135             uint8
forradrkommnr_0136             uint8
forradrkommnr_0137             uint8
forradrkommnr_0138             uint8
forradrkommnr_0211             uint8
forradrkommnr_0213             uint8
forradrkommnr_0214             uint8
forradrkommnr_0215             uint8
forradrkommnr_0216             uint8
forradrkommnr_0217             uint8
forradrkommnr_0219             uint8
f

In [24]:
df.head()

,ansatte_antall,regdato,forradrkommnr_0101,forradrkommnr_0104,forradrkommnr_0105,forradrkommnr_0106,forradrkommnr_0111,forradrkommnr_0118,forradrkommnr_0119,forradrkommnr_0121,...,sektorkode_4900,sektorkode_5500,sektorkode_5700,sektorkode_6100,sektorkode_6500,sektorkode_7000,sektorkode_8200,sektorkode_8300,sektorkode_8500,sektorkode_9000
0,76.0,1996-05-28,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1.0,2013-09-19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.0,2018-01-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,4.0,2013-08-24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
27,1.0,2014-06-19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
!pip install tables

    100% |################################| 3.8MB 431kB/s eta 0:00:01
    100% |################################| 163kB 8.1MB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
import tables

In [28]:
df.to_hdf('/storage/er_predict.h5', 'table')